In [1]:
from pathlib import Path
import pandas as pd
import logging

# Constantes
ROOT_PATH = Path.cwd().parent
DATA_RAW_FOLDER = ROOT_PATH / "data/raw"
DATA_PROCESSED_FOLDER = ROOT_PATH / "data/processed"
STRAVA_DATA_FILE = DATA_RAW_FOLDER / "strava_data.csv"
PROCESSED_DATA_FILE = DATA_PROCESSED_FOLDER / "data.csv"

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Dicionário de esportes em português
SPORT_TYPE_PT = {
    'Walk': 'Caminhada',
    'Run': 'Corrida',
    'StairStepper': 'Simulador de escadas',
    'WeightTraining': 'Treino com peso',
    'Workout': 'Treino livre',
    'Swim': 'Natação',
    'Ride': 'Bicicleta'
}

In [2]:
'''
 Salvando arquivo pronto para análise
'''
def salvar_dados(df, caminho):
    df.to_csv(caminho, index=False, header=True)
    logging.info(f"Arquivo {caminho.name} salvo com sucesso!")

In [3]:
'''
Substituindo os nomes das atividades fisicas equivalentes em português
'''
def substitui_sport_type(df):
    df['sport_type'] = df['sport_type'].replace(SPORT_TYPE_PT)
    logging.info("Sport_Type atualizado para PT-BR!")

In [4]:
'''
Atividades indoor: WeightTraining, Workout - Não há elevação
Atividades ao ar livre: Run, Walk Esteira - Elevação = 0
Max_Speed = Average_Speed
'''
def trata_nulos(df):
    atividades_indoor = ["WeightTraining", "Workout"]
    df.loc[df['sport_type'].isin(atividades_indoor), ['elev_high', 'elev_low']] = 0
    df.loc[(df['sport_type'] == 'Run') & (df['max_speed'] == 0), 'max_speed'] = df['average_speed']
    df.loc[(df['sport_type'] == 'Run') & (df['total_elevation_gain'] == 0), ['elev_high', 'elev_low']] = 0
    
    # Substituir valores nulos para 'Swim'
    valores_para_substituir = {'max_heartrate': 0, 'average_heartrate': 0}
    df.loc[df['sport_type'] == 'Swim', ['max_heartrate', 'average_heartrate']] = df.loc[df['sport_type'] == 'Swim', ['max_heartrate', 'average_heartrate']].fillna(valores_para_substituir)
    logging.info("Nulos tratados!")

In [5]:
'''
Verificando presença de valores nulos nos dados
'''
def verifica_nulos(df):
    nulos = df.isnull().sum()
    nulos = nulos[nulos > 0].to_dict()
    logging.info(f"Nulos encontrados: {nulos}")
    if nulos:
        trata_nulos(df)

    return df

In [6]:
''' 
- Dados a partir de 01-01-2022
- Distância em Km
- Velocidade em Km/h
- Criando coluna data apenas com dia, mês e ano
- Criando duração da atividade em segundos
- Criando coluna Ano
- Criando coluna Mês
'''
def prepara_dados(df):

    df_copy = df.copy()
    df_copy['start_date_local'] = pd.to_datetime(df_copy['start_date_local'])
    df_copy = df_copy[df_copy['start_date_local'] > '2022-01-01']
    df_copy['distance'] = df_copy['distance'] / 1000
    df_copy['average_speed'] = df_copy['average_speed'] * 3.6
    df_copy['max_speed'] = df_copy['max_speed'] * 3.6
    df_copy['data'] = df_copy['start_date_local'].dt.strftime('%d-%m-%Y')
    df_copy['duracao_atividade'] = df_copy['moving_time']
    df_copy['ano'] = df_copy['start_date_local'].dt.year
    df_copy['mes'] = df_copy['start_date_local'].dt.month
    logging.info("Dados preparados com sucesso!")

    return df_copy

In [7]:
# Ler do arquivo strava_data.csv
if not STRAVA_DATA_FILE.is_file():
    logging.warning(f"O arquivo {STRAVA_DATA_FILE.name} não foi encontrado! Por favor colete os dados do Strava novamente!")
else:
    df = pd.read_csv(STRAVA_DATA_FILE, encoding="utf-8")
    df = prepara_dados(df)
    verifica_nulos(df)
    substitui_sport_type(df)
    salvar_dados(df, PROCESSED_DATA_FILE)

2024-10-04 10:10:11,768 - INFO - Dados preparados com sucesso!
2024-10-04 10:10:11,772 - INFO - Nulos encontrados: {'average_heartrate': 11, 'max_heartrate': 11, 'elev_high': 63, 'elev_low': 63}
2024-10-04 10:10:11,784 - INFO - Nulos tratados!
2024-10-04 10:10:11,786 - INFO - Sport_Type atualizado para PT-BR!


2024-10-04 10:10:11,809 - INFO - Arquivo data.csv salvo com sucesso!
